# Mock Data - fastq
By Joely Nelson

Contact: joelyn@cs.washington.edu

The purpose of this file is to generate mock fastq files based on the mock bin data


The illumina sequencer does not produce a nice xlsx file with the sequences already nicely bins. Instead it will give us fastq files. This file is intended to use the binned files to creat mock fastq files.

#### Pipeline Visalization
![](.\\documentation\\mock_fastq_pipeline.png)

#### Parameters
This file takes in the following parameters:

* ``seq_name``: the file name to be read. It should have a column "ID" which is unique for every guide variant, and a "Sequence" column which contain's that guide's genetic sequence.
* ``barcode_fname``: the txt file that contains all the barcodes, where each barcode corresponds to each bin. Each barcode is on it's own line.
* ``bin_fname``: the file name that has the generated mock bins

#### Output

First, for each guide variant in the the associated read1 and read 2 are generated.

read 1 is (39 bases; begins with: TAGG (TATAATACTAGT) N20 GTT) and read2 is (36 bases: reverse complement of N20, reverse complement of (sequence) CCTA).

Then a unique barcode is assigned to each of the 10 bins.

Two fastq files are generated:

```read1_barcode_as_index.fastq```

and

```read2_barcode_as_index.fastq```

* Used the documentation provided here: https://support.illumina.com/help/BaseSpace_OLH_009008/Content/Source/Informatics/BS/FileFormat_FASTQ-files_swBS.htm to generate the files
* Each file of the two files contains 25980000 records. They have placeholders (like the phred score and run number and all that) for all elements except for:
    * read: is 1 or 2 depending on if it is read1 or read2
    * index: the barcode index associated with bin
    * sequence: the sequence of the guide variant based
    
#### Example    
Take poxBp_T7_TGT. It has a sequence of:

```
CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCTAGCTCAGTCCTAGGTATAATACTAGTTAACGGTTAAATAGCCCGATGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGT
```

It's read1 is calculated as: ``TAGGTATAATACTAGTTCCGTCAGATGAACTAAACTGTT``

It's read2 as: ``AGTTTAGTTCATCTGACGGAACTAGTATTATACCTA``

The second bin, bin 1, labeled 1000, is associated with ``AACCGCAA``. This is from the unique barcode file.

Based on the binning file, poxBp_T7_TGT has 32 reads in the second bin. So I generated 32 records for read1 and 32 records for read2. 

The 32 records in read1 look like this:

```
@SIM:1:FCX:1:14:6329:1045:1:N:0:AACCGCAA
TAGGTATAATACTAGTTCCGTCAGATGAACTAAACTGTT
+
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
```

and the 32 records in read2 look like this:

```
@SIM:1:FCX:1:14:6329:1045:2:N:0:AACCGCAA
AGTTTAGTTCATCTGACGGAACTAGTATTATACCTA
+
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
```

Note the read is the sequence, and the bin index goes in the index field of the header. The read field is different between the two files.

I did this for each of the guide variants and each of the bins to get a total of 25980000 records per file.

I was able to read the files with Biopythons SeqIO in the order of minutes.

#### Other Features of this Notebook
Provides several progress "bar" to show user how long the user has to wait. More of a sanity check to show the program isn't broken and is indeed making progress.

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from Bio import SeqIO 

## Parameters
In order for this code to run, you must provide a few parameters.
* ``seq_name``: the file name to be read. It should have a column ID which is unique for every guide variant.
* ``barcode_fname``: the txt file that contains all the barcodes. Each barcode is on it's own line.
* ``bin_fname``: the file name that has the generated mock bins

In [2]:
seq_fname = 'data/20191118-Sequences for synthesis_v2_VALIDATED.xlsx'
barcode_fname = 'data/8nt_index_dist4_barcode.txt'
bin_fname = 'data/mock_bins_counts.xlsx'

### Reading in Files
In this section, we read in the files and also show an example of the data we get

In [3]:
# df_seq
# a pandas df where we load the file of sequences
df_seq = pd.read_excel(seq_fname)
df_seq = df_seq.drop(columns=['Unnamed: 0'])

df_seq.head()

,Library,ID,Sequence,Spacer,PAM,Position,Strand
0,1_poxB,poxBp_T7_TGT,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,TCCGTCAGATGAACTAAACT,TGT,7.0,Top
1,1_poxB,poxBp_T17_GAA,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,CCCTTCCCCCTCCGTCAGAT,GAA,17.0,Top
2,1_poxB,poxBp_T18_TGA,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,TCCCTTCCCCCTCCGTCAGA,TGA,18.0,Top
3,1_poxB,poxBp_T20_GAT,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,CATCCCTTCCCCCTCCGTCA,GAT,20.0,Top
4,1_poxB,poxBp_T21_AGA,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,CCATCCCTTCCCCCTCCGTC,AGA,21.0,Top


In [4]:
#bar_lst
# a list where we keep the barcodes
f = open(barcode_fname, "r")
bar_list = f.read().split('\n')
f.close()

bar_list

['AACCAACG',
 'AACCGCAA',
 'AACCTGGC',
 'AACGACGT',
 'AACGCATC',
 'AACGTTAG',
 'AACTAGTA',
 'AACTCTCT',
 'AAGAACCA',
 'AAGACGAC',
 'AAGAGAGT',
 'AAGCATAT',
 'AAGCTATA',
 'AAGGCTGA',
 'AAGTCCTG',
 'AATAATGC',
 'AATAGGCG',
 'AATATCAT',
 'AATCCGTT',
 'AATTGAAC',
 'ACCAAGAG',
 'ACCAGACC',
 'ACCGTCCA',
 'ACCTCAGG',
 'ACCTGCTT',
 'ACGACTCG',
 'ACGATCGC',
 'ACGCAGTC',
 'ACGCGAAG',
 'ACGGAACT',
 'ACGTTGAT',
 'ACTATATG',
 'ACTCCTAC',
 'ACTGGAGA',
 'ACTTACCG',
 'AGAACGCT',
 'AGAAGCGG',
 'AGAATAAC',
 'AGACCATG',
 'AGAGATCG',
 'AGAGTCTT',
 'AGATAAGA',
 'AGATGGTC',
 'AGCAACTC',
 'AGCAGTAT',
 'AGCTTGCG',
 'AGGCGCCT',
 'AGGCTTGG',
 'AGTATGGA',
 'AGTTCCGC',
 'ATAACTTC',
 'ATACGAGC',
 'ATACTCAG',
 'ATAGAGAC',
 'ATATTACT',
 'ATGATTAA',
 'ATGGCCAT',
 'ATTAGCTA',
 'ATTCATTG',
 'ATTGGTCC',
 'CAACGGAC',
 'CAAGAGTT',
 'CAAGCAGG',
 'CAAGTCAA',
 'CAATGCCT',
 'CAATTATC',
 'CAGATTCC',
 'CAGCAAGC',
 'CAGCGTTG',
 'CAGTCGGT',
 'CATACCTC',
 'CATCTCCG',
 'CATTCTAA',
 'CCAACCAT',
 'CCAATTGG',
 'CCAGCGCC',
 'CCATACGC',

In [5]:
# df_bins
# a pandas df with the bins read in
df_bin = pd.read_excel(bin_fname)
df_bin = df_bin.drop(columns=['Unnamed: 0'])

df_bin.head()

,ID,0,1000,2000,3000,4000,5000,6000,7000,8000,9000
0,poxBp_T7_TGT,6,559,5172,7461,1741,61,0,0,0,0
1,poxBp_T17_GAA,0,0,3,96,1938,7422,4942,586,13,0
2,poxBp_T18_TGA,0,1,93,1298,5064,6162,2148,230,4,0
3,poxBp_T20_GAT,0,0,1,93,1576,6170,5734,1354,69,3
4,poxBp_T21_AGA,0,9,274,2858,6992,4239,606,22,0,0


## Helper Functions

In [6]:
def rev_comp(seq):
    '''
    Switches a seqence from top to bottom
    (ie 'ATGC' becomes 'TACG')    
    ARGS:
        -seq: (str) a string which represents the top strand of DNA.
               Should only have the characters ATGC
    RETURNS:
        string sequence that represents the bottom sequence
        (all Ts become As, As become Ts, Gs become Cs, Cs become Gs)
        and is revered
        Resulting string will be all uppercase
    '''
    res = seq.lower()
    res = res.replace('t', 'A')
    res = res.replace('a', 'T')
    res = res.replace('g', 'C')
    res = res.replace('c', 'G')
    res = res[::-1]
    return res


## Prepping data structures

In [7]:
# num bins:
# the number of bins that we used,
# should come from the df_bins file
# subtract the one to get rid of the label column
n_bins = len(list(df_bin.columns)) - 1

In [8]:
# barcodes: a list of barcodes that we are going to use
# we need as many barcodes as there are bins
barcodes = bar_list[:n_bins]

In [9]:
# df:
# join df_seq and df_bins on 'ID'
df = df_seq.merge(df_bin, left_on='ID', right_on='ID')
df

,Library,ID,Sequence,Spacer,PAM,Position,Strand,0,1000,2000,3000,4000,5000,6000,7000,8000,9000
0,1_poxB,poxBp_T7_TGT,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,TCCGTCAGATGAACTAAACT,TGT,7.0,Top,6,559,5172,7461,1741,61,0,0,0,0
1,1_poxB,poxBp_T17_GAA,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,CCCTTCCCCCTCCGTCAGAT,GAA,17.0,Top,0,0,3,96,1938,7422,4942,586,13,0
2,1_poxB,poxBp_T18_TGA,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,TCCCTTCCCCCTCCGTCAGA,TGA,18.0,Top,0,1,93,1298,5064,6162,2148,230,4,0
3,1_poxB,poxBp_T20_GAT,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,CATCCCTTCCCCCTCCGTCA,GAT,20.0,Top,0,0,1,93,1576,6170,5734,1354,69,3
4,1_poxB,poxBp_T21_AGA,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,CCATCCCTTCCCCCTCCGTC,AGA,21.0,Top,0,9,274,2858,6992,4239,606,22,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1727,25_J3,J3_B195_CGC,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,ATAAGCGCTGCGTGAATGAT,CGC,195.0,Bottom,0,83,1363,5877,6013,1569,94,1,0,0
1728,25_J3,J3_B197_CAA,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,AAGCGCTGCGTGAATGATCG,CAA,197.0,Bottom,0,3,495,5398,7681,1393,29,1,0,0
1729,25_J3,J3_B201_TGC,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,GCTGCGTGAATGATCGCAAA,TGC,201.0,Bottom,0,0,10,277,2954,7155,4063,526,15,0
1730,25_J3,J3_OT,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,GACTTGTGAAATGCGAGACC,NaN,NaN,NaN,0,0,2,194,3134,7945,3457,262,6,0


## Read Functions
We get the 2 reads from this

In [10]:
df.iloc[0, 2] #== 'CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCTAGCTCAGTCCTAGGTATAATACTAGTTCCGTCAGATGAACTAAACTGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGT'

'CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCTAGCTCAGTCCTAGGTATAATACTAGTTCCGTCAGATGAACTAAACTGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGT'

In [11]:
def get_read1(seq):
    '''
    ARGS:
        seq: A string ATCG sequence we will use
    Returns:
        a string with read1
        which is 39 bases; begins with: TAGG (TATAATACTAGT) N20 GTT 
    '''
    #  
    i = seq.find('TAGG')
    return seq[i:i+39]
    
seq = df.iloc[0, 2]
get_read1(seq)

'TAGGTATAATACTAGTTCCGTCAGATGAACTAAACTGTT'

In [12]:
def get_read2(seq):
    '''
    ARGS:
        seq: A string ATCG sequence we will use
    Returns:
        a string with read2
        which is 36 bases: reverse complement of N20, reverse complement of (sequence) CCTA
    '''
    seq_rev = rev_comp(seq)
    i = seq_rev.find('CCTA')
    return seq_rev[i-32:i+4]

get_read2(seq)

'AGTTTAGTTCATCTGACGGAACTAGTATTATACCTA'

In [13]:
len(get_read2(seq)), len('ACGGAGCGTTCTGGACACAAACTAGTATTATACCTA')

(36, 36)

Now we apply both read1 and read2 into the dataframe so that read1 and read2 for each sequence is known

In [14]:
df['read1'] = df['Sequence'].apply(get_read1)
df['read2'] = df['Sequence'].apply(get_read2)
df.head()

,Library,ID,Sequence,Spacer,PAM,Position,Strand,0,1000,2000,3000,4000,5000,6000,7000,8000,9000,read1,read2
0,1_poxB,poxBp_T7_TGT,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,TCCGTCAGATGAACTAAACT,TGT,7.0,Top,6,559,5172,7461,1741,61,0,0,0,0,TAGGTATAATACTAGTTCCGTCAGATGAACTAAACTGTT,AGTTTAGTTCATCTGACGGAACTAGTATTATACCTA
1,1_poxB,poxBp_T17_GAA,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,CCCTTCCCCCTCCGTCAGAT,GAA,17.0,Top,0,0,3,96,1938,7422,4942,586,13,0,TAGGTATAATACTAGTCCCTTCCCCCTCCGTCAGATGTT,ATCTGACGGAGGGGGAAGGGACTAGTATTATACCTA
2,1_poxB,poxBp_T18_TGA,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,TCCCTTCCCCCTCCGTCAGA,TGA,18.0,Top,0,1,93,1298,5064,6162,2148,230,4,0,TAGGTATAATACTAGTTCCCTTCCCCCTCCGTCAGAGTT,TCTGACGGAGGGGGAAGGGAACTAGTATTATACCTA
3,1_poxB,poxBp_T20_GAT,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,CATCCCTTCCCCCTCCGTCA,GAT,20.0,Top,0,0,1,93,1576,6170,5734,1354,69,3,TAGGTATAATACTAGTCATCCCTTCCCCCTCCGTCAGTT,TGACGGAGGGGGAAGGGATGACTAGTATTATACCTA
4,1_poxB,poxBp_T21_AGA,CTGAAGTCAGCCCCATACGATATAAGTTGTTACTAGATTGACAGCT...,CCATCCCTTCCCCCTCCGTC,AGA,21.0,Top,0,9,274,2858,6992,4239,606,22,0,0,TAGGTATAATACTAGTCCATCCCTTCCCCCTCCGTCGTT,GACGGAGGGGGAAGGGATGGACTAGTATTATACCTA


## Generate new data
We now generate a dataframe with: Read 1; Read 2; Bin Index; 

In [15]:
d_fastq = {'Read 1': [], 'Read 2': [], 'Bin Index': []}

for index, row in df.iterrows():
    # just for seeing progress:
    print(round((index + 1)/len(df) * 100, 1), "%")
    clear_output(wait=True)
    
    # for each sequence get read1 and read2
    read1 = row['read1']
    read2 = row['read2']
    
    # for each bin in the barcodes
    for b in range(n_bins):
        # get the number of reads for this bin
        num_reads = df.iloc[index, 7 + b]        
        barcode = barcodes[b]
        
        for r in range(num_reads):
            d_fastq['Read 1'].append(read1)
            d_fastq['Read 2'].append(read2)
            d_fastq['Bin Index'].append(barcode)
        

100.0 %


In [16]:
df_fastq = pd.DataFrame.from_dict(d_fastq)

In [17]:
df_fastq.head()

,Read 1,Read 2,Bin Index
0,TAGGTATAATACTAGTTCCGTCAGATGAACTAAACTGTT,AGTTTAGTTCATCTGACGGAACTAGTATTATACCTA,AACCAACG
1,TAGGTATAATACTAGTTCCGTCAGATGAACTAAACTGTT,AGTTTAGTTCATCTGACGGAACTAGTATTATACCTA,AACCAACG
2,TAGGTATAATACTAGTTCCGTCAGATGAACTAAACTGTT,AGTTTAGTTCATCTGACGGAACTAGTATTATACCTA,AACCAACG
3,TAGGTATAATACTAGTTCCGTCAGATGAACTAAACTGTT,AGTTTAGTTCATCTGACGGAACTAGTATTATACCTA,AACCAACG
4,TAGGTATAATACTAGTTCCGTCAGATGAACTAAACTGTT,AGTTTAGTTCATCTGACGGAACTAGTATTATACCTA,AACCAACG


---
---

# To Fastq

We look at some exmples of fastq datafiles to figure out what we would need to export.

#### Exploring Ban Wang's fastq data

In [18]:
fname = "data/501_0_S27_R2_001.fastq"
for r in SeqIO.parse(fname, "fastq"):
    break
# print(r, '\n')
# print(r.seq)
# print(r.id)
# type(r.id)
r.id.split(":")

['NB501203', '285', 'H2TKCBGXC', '1', '11101', '6053', '1067']

In [19]:
f = open(fname, "r")
i = 0
for line in f:
    print(line, end = "")
    if i == 7:
        break
    i += 1
f.close()

@NB501203:285:H2TKCBGXC:1:11101:6053:1067 2:N:0:TCGCCTTA+TCATGAGC
GAACGATNNN
+
AAAAAEE###
@NB501203:285:H2TKCBGXC:1:11101:18448:1067 2:N:0:TCGCCTTA+TCATGAGC
CCGAAAGNNN
+
AAAAAEE###


Looking [here](https://support.illumina.com/help/BaseSpace_OLH_009008/Content/Source/Informatics/BS/FileFormat_FASTQ-files_swBS.htm) for documentation on how the illumina file returns data

In [20]:
record = "@SIM:1:FCX:1:15:6329:1045:GATTACT+GTCTTAAC 1:N:0:ATCCGA\nTCGCACTCAACGCCCTGCATATGACAAGACAGAATC\n+\n<>;##=><9=AAAAAAAAAA9#:<#<;<<<????#="
print("Raw Record:\n")
print(record)

Raw Record:

@SIM:1:FCX:1:15:6329:1045:GATTACT+GTCTTAAC 1:N:0:ATCCGA
TCGCACTCAACGCCCTGCATATGACAAGACAGAATC
+
<>;##=><9=AAAAAAAAAA9#:<#<;<<<????#=


In [21]:
print("Record Lines:\n")
for line, label in zip(record.split("\n"), ['Sequence ID', 'Sequence', 'Quality ID', 'Quality score']):
    print(label, "\t", line)

Record Lines:

Sequence ID 	 @SIM:1:FCX:1:15:6329:1045:GATTACT+GTCTTAAC 1:N:0:ATCCGA
Sequence 	 TCGCACTCAACGCCCTGCATATGACAAGACAGAATC
Quality ID 	 +
Quality score 	 <>;##=><9=AAAAAAAAAA9#:<#<;<<<????#=


In [22]:
print("Sequence ID Elements:\n")
seqid = record.split("\n")[0]
labels = ['instrument', 'run num', 'flowcell id', 'lane\t', 'title\t', 'xpos\t', 'ypos\t', 'UMI/Read', 'is filtered', 'control num', 'index\t']
for e, label in zip(seqid.split(":"), labels):
    print(label, "\t", e)

Sequence ID Elements:

instrument 	 @SIM
run num 	 1
flowcell id 	 FCX
lane	 	 1
title	 	 15
xpos	 	 6329
ypos	 	 1045
UMI/Read 	 GATTACT+GTCTTAAC 1
is filtered 	 N
control num 	 0
index	 	 ATCCGA


### Helper Functions
Below we have
* gen_record: creates a record for the illumina machine given the fields
* parse_id: parses the id of a record

In [23]:
def gen_record(instrument = 'SIM',
               run_num = '1',
               flowcell_id = 'FCX',
               lane = '1',
               title = '14',
               xpos = '6329',
               ypos = '1045',
               UMI = '',
               read = '',
               is_filtered = 'N',
               control_num = '0',
               index = '',
               seq = '',
               quality_score = None
              ):
    '''
    given the elements of a fastq record will generate it.
    ARGS:
        - instrument: Instrument ID. Characters allowed: a–z, A–Z, 0–9 and underscore
        - run_num: numerical. Run number on instrument
        - flow_cell_id: numerical
        - lane: numerical. Lane number.
        - title: numerical. Title number.
        - x_pos: numerical. X coordinate of cluster.
        - y_pos: numerical. Y coordinate of cluster.
        - UMI: sequence (ATGCN). UMI sequences for read 1 and read2 seperated by +
        - read: numerical. Read number. 1 can be single read or Read 2 of paired-end
        - is_filtered: Y or N. Y if the read is filtered (did not pass), N otherwise
        - control_number: numerical. 0 when none of the control bits are on, otherwise it is an even number.
        - index: (ATGCN). index of the read
        - seq: (ATGCN). The sequence read
        - quality_score: the quality score. If it is left as None, will omit the +
    RETURN:
        Returns a 2 or 4 lined string that represents how this record would be presented in a fastq file
    '''
    # make sure everything is a string or in the appropraite form
    instrument = str(instrument)
    run_num = str(run_num)
    flowcell_id = str(flowcell_id)
    lane = str(lane)
    title = str(title)
    xpos = str(xpos)
    ypos = str(ypos)
    UMI = str(UMI)
    read = str(read)
    if type(is_filtered) == 'bool':
        if is_filtered:
            is_filtered = 'Y'
        else:
            is_filtered = 'N'
    is_filtered = str(is_filtered)
    control_num = str(control_num)
    index = str(index)
    seq = str(seq)
    
    line1 = "@" + instrument + ":" + run_num + ':' + flowcell_id + ':' + lane + ':' + title + ':' + xpos + ':' + ypos + ':' + UMI + read  + ':' + is_filtered + ':' + control_num + ':' + index + "\n"
    line2 = seq + "\n"
    line3 = "" 
    line4 = ""
    if not (quality_score is None):
        line3 = "+\n"
        line4 = str(quality_score) + "\n"
    return line1 + line2 + line3 + line4


In [24]:
def parse_id(record):
    '''
    Given a whole record (or just the id)
    parses it and returns it as a dictionary
    ARGS:
        - record: (str) a string of the record or just the id
    RETURNS:
        a dictionary with the following keys:
        - instrument: Instrument ID. Characters allowed: a–z, A–Z, 0–9 and underscore
        - run_num: numerical. Run number on instrument
        - flow_cell_id: numerical
        - lane: numerical. Lane number.
        - title: numerical. Title number.
        - x_pos: numerical. X coordinate of cluster.
        - y_pos: numerical. Y coordinate of cluster.
        - UMI: sequence (ATGCN). UMI sequences for read 1 and read2 seperated by +
        - read: numerical. Read number. 1 can be single read or Read 2 of paired-end
        - is_filtered: Y or N. Y if the read is filtered (did not pass), N otherwise
        - control_number: numerical. 0 when none of the control bits are on, otherwise it is an even number.
        - index: (ATGCN). index of the read
    '''
    seqid = record.split("\n")[0]
    labels = ['instrument', 'run num', 'flowcell_id', 'lane', 'title', 'xpos', 'ypos', 'UMI/read', 'is_filtered', 'control_num', 'index']
    d = dict()
    for e, label in zip(seqid.split(":"), labels):
        if label == 'UMI/read':
            umi, read = e.split(' ')
            d['UMI'] = umi
            d['read'] = read
        else:
            d[label] = e
    return d
     
parse_id(record)

{'instrument': '@SIM',
 'run num': '1',
 'flowcell_id': 'FCX',
 'lane': '1',
 'title': '15',
 'xpos': '6329',
 'ypos': '1045',
 'UMI': 'GATTACT+GTCTTAAC',
 'read': '1',
 'is_filtered': 'N',
 'control_num': '0',
 'index': 'ATCCGA'}

In our data...
* Sequence ID
    * instrument: @SIM
    * run num: NA
    * flowcell id: NA
    * lane: NA
    * title: NA
    * xpos: NA
    * ypos: NA
    * UMI/Read: ?!?!?!?!?!?!?!?!?!?!?!?!?!!??!!??!?!?!
    * is filtered: NA
    * control num: NA
    * index: ?!?!?!?!?!?!?!?!?!?!?!?!?!!??!!??!?!?!
* Sequence
    * The sequence it read
* Quality score id line (+)
* Quality score
    * placeholder

In [25]:
fname1 = "data/read1_barcode_as_index.fastq"
fname2 = "data/read2_barcode_as_index.fastq"

f1 = open(fname1, "w")
f2 = open(fname2, "w")

d_fastq = {'Read 1': [], 'Read 2': [], 'Bin Index': []}

for index, row in df.iterrows():
    # just for seeing progress:
    print(round(index/len(df) * 100, 1), "%")
    clear_output(wait=True)
    
    # for each sequence get read1 and read2
    read1 = row['read1']
    read2 = row['read2']
    
    # generate random qs
    qs1 = "A" * len(read1) 
    qs2 = "A" * len(read2)
        
    # for each bin in the barcodes
    for b in range(n_bins):
        # get the number of reads for this bin
        num_reads = df.iloc[index, 7 + b]        
        barcode = barcodes[b]       
        
        # generate the records
        # using defaults provided in the documentation
        r1 = gen_record(read = 1, index = barcode, seq = read1, quality_score = qs1)
        r2 = gen_record(read = 2, index = barcode, seq = read2, quality_score = qs2)
        
        for r in range(num_reads):
            # write into the two files
            f1.write(r1)
            f2.write(r2)
            
f1.close()
f2.close()

99.9 %


## Testing to see if that worked

In [26]:
r1, r2

('@SIM:1:FCX:1:14:6329:1045:1:N:0:AAGACGAC\nTAGGGCGTGTACTAGTGACTTGTGAAATGCGAGACCGTT\n+\nAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\n',
 '@SIM:1:FCX:1:14:6329:1045:2:N:0:AAGACGAC\nGGTCTCGCATTTCACAAGTCACTAGTACACGCCCTA\n+\nAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\n')

In [27]:
i = 0
for r in SeqIO.parse(fname1, "fastq"):
    print(r)
    i += 1
    if i == 1:
        break

# number of records: 25980000

ID: SIM:1:FCX:1:14:6329:1045:1:N:0:AACCAACG
Name: SIM:1:FCX:1:14:6329:1045:1:N:0:AACCAACG
Description: SIM:1:FCX:1:14:6329:1045:1:N:0:AACCAACG
Number of features: 0
Per letter annotation for: phred_quality
Seq('TAGGTATAATACTAGTTCCGTCAGATGAACTAAACTGTT')


In [28]:
f = open(fname2, "r")
i = 0
for line in f:
    print(line, end = "")
    if i == 3:
        break
    i += 1
f.close()

@SIM:1:FCX:1:14:6329:1045:2:N:0:AACCAACG
AGTTTAGTTCATCTGACGGAACTAGTATTATACCTA
+
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
